# Init

In [2]:
cd ..

/Users/wliao0504/code/clif/CLIF-MIMIC


/Users/wliao0504/code/clif/CLIF-MIMIC/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [8]:
pwd

'/Users/wliao0504/code/clif/CLIF-MIMIC'

In [3]:
# src/tables/patient_assessments.py
import numpy as np
import pandas as pd
import logging
import duckdb
from importlib import reload
import src.utils
reload(src.utils)
from src.utils import construct_mapper_dict, fetch_mimic_events, load_mapping_csv, \
    get_relevant_item_ids, find_duplicates, rename_and_reorder_cols, save_to_rclif, \
    convert_and_sort_datetime, setup_logging, con, REPO_ROOT, mimic_table_pathfinder, \
    search_mimic_items, clif_table_pathfinder

loaded configuration from /Users/wliao0504/code/clif/CLIF-MIMIC/src/../config/config.json
loaded configuration from /Users/wliao0504/code/clif/CLIF-MIMIC/src/../config/config.json


In [4]:
setup_logging()

GCS_MAPPER = {"gcs": "gcs_total", "gcs_eyes": "gcs_eye"}

PA_COL_NAMES = [
    "hospitalization_id", "recorded_dttm", "assessment_name", "assessment_category",
    "assessment_group", "numerical_value", "categorical_value", "text_value"
    ]

PA_COLS_RENAME_MAPPER = {
    "time": "recorded_dttm", "value": "text_value"
}

2025-04-22 23:19:52,132 - INFO - initialized logging at logs/etl.log


## Pipeline

In [64]:
import importlib
from src.tables import patient_assessments 
importlib.reload(patient_assessments)

from src import utils
importlib.reload(utils)

2025-04-26 22:01:00,559 - INFO - initialized logging at logs/etl.log


loaded configuration from /Users/wliao0504/code/clif/CLIF-MIMIC/src/../config/config.json


<module 'src.utils' from '/Users/wliao0504/code/clif/CLIF-MIMIC/src/utils.py'>

# Dev

In [65]:
print("testing all...")
from hamilton import driver
setup_logging()
dr = (
    driver.Builder()
    .with_modules(patient_assessments)
    .build()
)
all_possible_outputs = dr.list_available_variables()
desired_outputs = [o.name for o in all_possible_outputs
                if 'test' == o.tags.get('property')]
output = dr.execute(desired_outputs)

2025-04-26 22:01:02,447 - INFO - initialized logging at logs/etl.log
2025-04-26 22:01:02,449 - INFO - fetching SBT data...
2025-04-26 22:01:02,449 - INFO - querying the d_items table to identify which event tables to be separately queried for 3 items
2025-04-26 22:01:02,452 - INFO - identified 1 event tables to be separately queried: ['chartevents']
2025-04-26 22:01:02,453 - INFO - fetching events from chartevents table for 3 items


testing all...


2025-04-26 22:01:04,401 - INFO - fetched 65230 events from chartevents table for 3 items
2025-04-26 22:01:04,401 - INFO - concatenated 65230 events from 1 event tables
2025-04-26 22:01:04,535 - INFO - executing official MIMIC script to fetch GCS data...


testing schema...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

2025-04-26 22:01:39,371 - INFO - fetching CAM data...
2025-04-26 22:01:39,373 - INFO - querying the d_items table to identify which event tables to be separately queried for 11 items
2025-04-26 22:01:39,377 - INFO - identified 1 event tables to be separately queried: ['chartevents']
2025-04-26 22:01:39,377 - INFO - fetching events from chartevents table for 11 items
2025-04-26 22:01:41,460 - INFO - fetched 855297 events from chartevents table for 11 items
2025-04-26 22:01:41,461 - INFO - concatenated 855297 events from 1 event tables


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

2025-04-26 22:01:46,025 - INFO - merging all of the above...
2025-04-26 22:01:46,920 - INFO - converting column dtypes...


testing schema...


In [66]:
output

{'sbt_tested': True, 'schema_tested': True}

In [54]:
fail_data.value_counts(["assessment_name", "assessment_category"], dropna=False)

assessment_name                assessment_category   
gcs                            gcs_total                 2217787
gcs_verbal                     gcs_verbal                2217787
gcs_motor                      gcs_motor                 2217787
gcs_eyes                       gcs_eye                   2217787
Richmond-RAS Scale             RASS                      1656001
Braden Sensory Perception      braden_sensory            1068778
Braden Moisture                braden_moisture           1068127
Braden Activity                braden_activity           1067495
Braden Mobility                braden_mobility           1067493
Braden Nutrition               braden_nutrition          1067041
Braden Friction/Shear          braden_friction           1063850
NaN                            braden_total              1061624
CAM-ICU MS Change              cam_mental                 518056
NaN                            cam_total                  477061
CAM-ICU Inattention            cam_i

In [53]:
fail_data.loc[fail_data["assessment_name"].isna()].value_counts("assessment_category")

assessment_category
braden_total    1061624
cam_total        477061
Name: count, dtype: int64

## Braden

In [54]:
braden = fetch_mimic_events([224054, 224055, 224056, 224057, 224058, 224059])

2025-01-20 20:41:12,588 - INFO - querying the d_items table to identify which event tables to be separately queried for 6 items
2025-01-20 20:41:12,593 - INFO - identified 1 event tables to be separately queried: ['chartevents']
2025-01-20 20:41:12,593 - INFO - fetching events from chartevents table for 6 items
2025-01-20 20:41:15,872 - INFO - fetched 6402784 events from chartevents table for 6 items
2025-01-20 20:41:15,873 - INFO - concatenated 6402784 events from 1 event tables


In [55]:
braden_eg = braden.query("hadm_id == 25860671").drop(columns=["stay_id", "valueuom", "itemid"])
braden_eg

,label,hadm_id,time,value
0,Braden Sensory Perception,25860671,2150-11-05 08:00:00,Slight Impairment
1,Braden Moisture,25860671,2150-11-05 08:00:00,Occasionally Moist
2,Braden Activity,25860671,2150-11-05 08:00:00,Bedfast
3,Braden Mobility,25860671,2150-11-05 08:00:00,Very Limited
4,Braden Friction/Shear,25860671,2150-11-05 08:00:00,Potential Problem
...,...,...,...,...
1171,Braden Moisture,25860671,2150-11-04 04:00:00,Occasionally Moist
1172,Braden Activity,25860671,2150-11-04 04:00:00,Bedfast
1173,Braden Mobility,25860671,2150-11-04 04:00:00,Very Limited
1174,Braden Friction/Shear,25860671,2150-11-04 04:00:00,Potential Problem


In [56]:
# this can be a potential test
query = f"""
PIVOT braden
ON label
USING COUNT(value)
GROUP BY hadm_id, time
"""
braden_w = con.execute(query).fetchdf()
braden_w

,hadm_id,time,Braden Activity,Braden Friction/Shear,Braden Mobility,Braden Moisture,Braden Nutrition,Braden Sensory Perception
0,23822395,2129-08-08 00:00:00,1,1,1,1,1,1
1,23822395,2129-08-09 20:00:00,1,1,1,1,1,1
2,27882036,2176-11-27 20:00:00,1,1,1,1,1,1
3,27882036,2176-11-29 08:00:00,1,1,1,1,1,1
4,28662225,2156-04-16 12:00:00,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...
1070265,21614463,2167-08-26 08:00:00,1,1,1,1,1,1
1070266,21614463,2167-08-28 13:22:00,1,1,1,1,1,1
1070267,25166113,2111-10-08 20:00:00,1,1,1,1,1,1
1070268,25166113,2111-10-09 20:00:00,1,1,1,1,1,1


In [57]:
query = f"""
PIVOT braden
ON label
USING FIRST(value)
GROUP BY hadm_id, time
"""
braden_w = con.execute(query).fetchdf()
braden_w

,hadm_id,time,Braden Activity,Braden Friction/Shear,Braden Mobility,Braden Moisture,Braden Nutrition,Braden Sensory Perception
0,22928422,2182-09-06 16:00:00,Bedfast,Potential Problem,Slight Limitations,Occasionally Moist,Adequate,Slight Impairment
1,25054284,2148-08-15 20:01:00,Bedfast,No Apparent Problem,Slight Limitations,Rarely Moist,Probably Inadequate,Slight Impairment
2,25525077,2129-03-13 04:00:00,Bedfast,Potential Problem,Completely Immobile,Rarely Moist,Probably Inadequate,Completely Limited
3,25525077,2129-03-07 12:00:00,Bedfast,Potential Problem,Very Limited,Rarely Moist,Probably Inadequate,Very Limited
4,29960393,2173-09-25 11:00:00,Walks Occasionally,No Apparent Problem,Slight Limitations,Rarely Moist,Adequate,No Impairment
...,...,...,...,...,...,...,...,...
1070265,25893125,2148-06-05 20:00:00,Bedfast,Potential Problem,Slight Limitations,Occasionally Moist,Adequate,Slight Impairment
1070266,23974045,2125-02-24 16:23:00,Bedfast,Problem,Very Limited,Occasionally Moist,Adequate,No Impairment
1070267,23974045,2125-02-23 00:12:00,Bedfast,Problem,Very Limited,Occasionally Moist,Adequate,No Impairment
1070268,21191593,2133-04-20 00:00:00,Chairfast,Potential Problem,Slight Limitations,Rarely Moist,Adequate,Slight Impairment


In [58]:
braden_w.value_counts("Braden Sensory Perception")

Braden Sensory Perception
Slight Impairment     429128
No Impairment         303227
Very Limited          243696
Completely Limited     92727
Name: count, dtype: int64

In [59]:
query = f"""
SELECT
    hadm_id, time, 
    CASE
        WHEN "Braden Activity" = 'Bedfast' THEN 1
        WHEN "Braden Activity" = 'Chairfast' THEN 2
        WHEN "Braden Activity" = 'Walks Occasionally' THEN 3
        WHEN "Braden Activity" = 'Walks Frequently' THEN 4
        ELSE NULL
        END AS braden_activity,
    CASE
        WHEN "Braden Friction/Shear" = 'Problem' THEN 1
        WHEN "Braden Friction/Shear" = 'Potential Problem' THEN 2
        WHEN "Braden Friction/Shear" = 'No Apparent Problem' THEN 3
        ELSE NULL
        END AS braden_friction,
    CASE
        WHEN "Braden Mobility" = 'Completely Immobile' THEN 1
        WHEN "Braden Mobility" = 'Very Limited' THEN 2
        WHEN "Braden Mobility" = 'Slight Limitations' THEN 3
        WHEN "Braden Mobility" = 'No Limitations' THEN 4
        ELSE NULL
        END AS braden_mobility,
    CASE
        WHEN "Braden Moisture" = 'Consistently Moist' THEN 1
        WHEN "Braden Moisture" = 'Moist' THEN 2
        WHEN "Braden Moisture" = 'Occasionally Moist' THEN 3
        WHEN "Braden Moisture" = 'Rarely Moist' THEN 4
        ELSE NULL
        END AS braden_moisture,
    CASE
        WHEN "Braden Nutrition" = 'Very Poor' THEN 1
        WHEN "Braden Nutrition" = 'Probably Inadequate' THEN 2
        WHEN "Braden Nutrition" = 'Adequate' THEN 3
        WHEN "Braden Nutrition" = 'Excellent' THEN 4
        ELSE NULL
        END AS braden_nutrition,
    CASE
        WHEN "Braden Sensory Perception" = 'Completely Limited' THEN 1
        WHEN "Braden Sensory Perception" = 'Very Limited' THEN 2
        WHEN "Braden Sensory Perception" = 'Slight Impairment' THEN 3
        WHEN "Braden Sensory Perception" = 'No Impairment' THEN 4
        ELSE NULL
        END AS braden_sensory,
    (braden_activity + braden_friction + braden_mobility + braden_moisture + braden_nutrition + braden_sensory) AS braden_total
FROM braden_w
"""
braden_wc = con.execute(query).fetchdf()
braden_wc

,hadm_id,time,braden_activity,braden_friction,braden_mobility,braden_moisture,braden_nutrition,braden_sensory,braden_total
0,22928422,2182-09-06 16:00:00,1.0,2.0,3.0,3.0,3.0,3.0,15.0
1,25054284,2148-08-15 20:01:00,1.0,3.0,3.0,4.0,2.0,3.0,16.0
2,25525077,2129-03-13 04:00:00,1.0,2.0,1.0,4.0,2.0,1.0,11.0
3,25525077,2129-03-07 12:00:00,1.0,2.0,2.0,4.0,2.0,2.0,13.0
4,29960393,2173-09-25 11:00:00,3.0,3.0,3.0,4.0,3.0,4.0,20.0
...,...,...,...,...,...,...,...,...,...
1070265,25893125,2148-06-05 20:00:00,1.0,2.0,3.0,3.0,3.0,3.0,15.0
1070266,23974045,2125-02-24 16:23:00,1.0,1.0,2.0,3.0,3.0,4.0,14.0
1070267,23974045,2125-02-23 00:12:00,1.0,1.0,2.0,3.0,3.0,4.0,14.0
1070268,21191593,2133-04-20 00:00:00,2.0,2.0,3.0,4.0,3.0,3.0,17.0


In [60]:
# unpivot (from wide to long) for the numerical values
query = f"""
UNPIVOT braden_wc
ON COLUMNS('braden_')
INTO
    NAME assessment_category
    VALUE numerical_value;
"""
braden_wcl = con.execute(query).fetchdf()
braden_wcl

,hadm_id,time,assessment_category,numerical_value
0,22928422,2182-09-06 16:00:00,braden_activity,1.0
1,22928422,2182-09-06 16:00:00,braden_friction,2.0
2,22928422,2182-09-06 16:00:00,braden_mobility,3.0
3,22928422,2182-09-06 16:00:00,braden_moisture,3.0
4,22928422,2182-09-06 16:00:00,braden_nutrition,3.0
...,...,...,...,...
7464403,29308245,2178-03-30 21:00:00,braden_mobility,4.0
7464404,29308245,2178-03-30 21:00:00,braden_moisture,4.0
7464405,29308245,2178-03-30 21:00:00,braden_nutrition,3.0
7464406,29308245,2178-03-30 21:00:00,braden_sensory,4.0


In [61]:
# unpivot (from wide to long) for the *categorical* values
query = f"""
UNPIVOT (
    SELECT
        hadm_id, time,
        "Braden Activity" AS braden_activity,
        "Braden Friction/Shear" AS braden_friction,
        "Braden Mobility" AS braden_mobility,
        "Braden Moisture" AS braden_moisture,
        "Braden Nutrition" AS braden_nutrition,
        "Braden Sensory Perception" AS braden_sensory
    FROM braden_w
)
ON COLUMNS('braden_')
INTO
    NAME assessment_category
    VALUE categorical_value;
"""
braden_wl = con.execute(query).fetchdf()
braden_wl

,hadm_id,time,assessment_category,categorical_value
0,22928422,2182-09-06 16:00:00,braden_activity,Bedfast
1,22928422,2182-09-06 16:00:00,braden_friction,Potential Problem
2,22928422,2182-09-06 16:00:00,braden_mobility,Slight Limitations
3,22928422,2182-09-06 16:00:00,braden_moisture,Occasionally Moist
4,22928422,2182-09-06 16:00:00,braden_nutrition,Adequate
...,...,...,...,...
6402779,29308245,2178-03-30 21:00:00,braden_friction,No Apparent Problem
6402780,29308245,2178-03-30 21:00:00,braden_mobility,No Limitations
6402781,29308245,2178-03-30 21:00:00,braden_moisture,Rarely Moist
6402782,29308245,2178-03-30 21:00:00,braden_nutrition,Adequate


In [62]:
# join the two long tables
braden_m = pd.merge(braden_wcl, braden_wl, on=["hadm_id", "time", "assessment_category"], how="outer")
braden_m

,hadm_id,time,assessment_category,numerical_value,categorical_value
0,20000094,2150-03-02 16:01:00,braden_activity,2.0,Chairfast
1,20000094,2150-03-02 16:01:00,braden_friction,2.0,Potential Problem
2,20000094,2150-03-02 16:01:00,braden_mobility,2.0,Very Limited
3,20000094,2150-03-02 16:01:00,braden_moisture,3.0,Occasionally Moist
4,20000094,2150-03-02 16:01:00,braden_nutrition,2.0,Probably Inadequate
...,...,...,...,...,...
7464403,29999828,2180-10-29 08:29:00,braden_mobility,4.0,No Limitations
7464404,29999828,2180-10-29 08:29:00,braden_moisture,3.0,Occasionally Moist
7464405,29999828,2180-10-29 08:29:00,braden_nutrition,2.0,Probably Inadequate
7464406,29999828,2180-10-29 08:29:00,braden_sensory,4.0,No Impairment


In [70]:
query = f"""
SELECT
    hadm_id as hospitalization_id,
    time as recorded_dttm,
    CASE
        WHEN assessment_category = 'braden_total' THEN NULL
        WHEN assessment_category = 'braden_activity' THEN 'Braden Activity'
        WHEN assessment_category = 'braden_friction' THEN 'Braden Friction/Shear'
        WHEN assessment_category = 'braden_mobility' THEN 'Braden Mobility'
        WHEN assessment_category = 'braden_moisture' THEN 'Braden Moisture'
        WHEN assessment_category = 'braden_nutrition' THEN 'Braden Nutrition'
        WHEN assessment_category = 'braden_sensory' THEN 'Braden Sensory Perception'
        ELSE assessment_category
        END AS assessment_name,
    assessment_category,
    numerical_value,
    categorical_value
FROM braden_m
"""
braden_mf = con.execute(query).fetchdf()
braden_mf

,hospitalization_id,recorded_dttm,assessment_name,assessment_category,numerical_value,categorical_value
0,20000094,2150-03-02 16:01:00,Braden Activity,braden_activity,2.0,Chairfast
1,20000094,2150-03-02 16:01:00,Braden Friction/Shear,braden_friction,2.0,Potential Problem
2,20000094,2150-03-02 16:01:00,Braden Mobility,braden_mobility,2.0,Very Limited
3,20000094,2150-03-02 16:01:00,Braden Moisture,braden_moisture,3.0,Occasionally Moist
4,20000094,2150-03-02 16:01:00,Braden Nutrition,braden_nutrition,2.0,Probably Inadequate
...,...,...,...,...,...,...
7464403,29999828,2180-10-29 08:29:00,Braden Mobility,braden_mobility,4.0,No Limitations
7464404,29999828,2180-10-29 08:29:00,Braden Moisture,braden_moisture,3.0,Occasionally Moist
7464405,29999828,2180-10-29 08:29:00,Braden Nutrition,braden_nutrition,2.0,Probably Inadequate
7464406,29999828,2180-10-29 08:29:00,Braden Sensory Perception,braden_sensory,4.0,No Impairment


## CAM-ICU

In [6]:
cam_icu = fetch_mimic_events(
    [228300, 228337, 229326, 228301, 228336, 229325, 228302, 228334, 228303, 228335, 229324]
)
cam_icu

2025-01-20 19:26:21,557 - INFO - querying the d_items table to identify which event tables to be separately queried for 11 items
2025-01-20 19:26:21,575 - INFO - identified 1 event tables to be separately queried: ['chartevents']
2025-01-20 19:26:21,576 - INFO - fetching events from chartevents table for 11 items
2025-01-20 19:26:23,477 - INFO - fetched 855297 events from chartevents table for 11 items
2025-01-20 19:26:23,478 - INFO - concatenated 855297 events from 1 event tables


,itemid,label,hadm_id,stay_id,time,value,valueuom
0,228337,CAM-ICU MS Change,29079034,39553978,2180-07-23 18:26:00,Yes,None
1,228337,CAM-ICU MS Change,29079034,39553978,2180-07-23 21:12:00,Yes,None
2,228337,CAM-ICU MS Change,26913865,39765666,2189-06-27 09:09:00,No,None
3,228337,CAM-ICU MS Change,24597018,37067082,2157-11-21 21:29:00,No,None
4,228337,CAM-ICU MS Change,24597018,37067082,2157-11-21 08:04:00,No,None
...,...,...,...,...,...,...,...
855292,228334,CAM-ICU Altered LOC,29058999,38050957,2182-01-20 20:00:00,Yes,None
855293,228300,CAM-ICU MS change,25208232,36790734,2134-08-09 08:00:00,Unable to Assess (Stop),None
855294,228300,CAM-ICU MS change,25208232,36790734,2134-08-10 08:00:00,Unable to Assess (Stop),None
855295,228300,CAM-ICU MS change,25208232,36790734,2134-08-10 00:00:00,No (Stop - Not delirious),None


In [7]:
# this can be a potential test
query = f"""
PIVOT (
    SELECT
        hadm_id, time,
        itemid,
        CASE
            WHEN itemid = 228300 THEN 'CAM-ICU MS Change'
            WHEN itemid in (228302, 228334) THEN 'CAM-ICU LOC'
            ELSE label
        END AS label,
        value
    FROM cam_icu
)
ON label
USING COUNT(value)
GROUP BY hadm_id, time
"""
cam_icu_w = con.execute(query).fetchdf()
cam_icu_w

,hadm_id,time,CAM-ICU Disorganized thinking,CAM-ICU Inattention,CAM-ICU LOC,CAM-ICU MS Change
0,28192338,2137-11-11 04:29:00,0,0,0,1
1,28192338,2137-11-12 12:00:00,0,0,0,1
2,28192338,2137-11-12 16:00:00,0,0,0,1
3,28192338,2137-11-13 12:00:00,0,0,0,1
4,21380829,2159-07-14 08:00:00,0,0,0,1
...,...,...,...,...,...,...
526884,21512906,2136-04-10 20:00:00,0,0,0,1
526885,29355211,2165-04-11 08:00:00,0,0,0,1
526886,29355211,2165-04-12 16:00:00,0,0,0,1
526887,29355211,2165-04-09 08:03:00,0,0,0,1


In [36]:
query = f"""
PIVOT (
    SELECT
        hadm_id, time,
        itemid,
        CASE
            WHEN itemid = 228300 THEN 'CAM-ICU MS Change'
            ELSE label
        END AS label,
        value
    FROM cam_icu
)
ON label
USING FIRST(value)
GROUP BY hadm_id, time
"""
cam_icu_w = con.execute(query).fetchdf()
cam_icu_w.head()

,hadm_id,time,CAM-ICU Altered LOC,CAM-ICU Disorganized thinking,CAM-ICU Inattention,CAM-ICU MS Change,CAM-ICU RASS LOC
0,28192338,2137-11-11 20:00:00,None,None,None,No,None
1,26789314,2161-04-15 00:00:00,None,None,None,No,None
2,21380829,2159-08-29 22:38:00,None,None,None,No,None
3,21380829,2159-08-29 08:00:00,None,None,None,No,None
4,21380829,2159-07-22 20:00:00,None,None,None,No,None


In [38]:
print(cam_icu_w.value_counts("CAM-ICU MS Change"))
print(cam_icu_w.value_counts("CAM-ICU Inattention"))
print(cam_icu_w.value_counts("CAM-ICU Disorganized thinking"))
print(cam_icu_w.value_counts("CAM-ICU Altered LOC"))
print(cam_icu_w.value_counts("CAM-ICU RASS LOC"))

CAM-ICU MS Change
No                           319063
Yes                          177720
Unable to Assess              14723
No (Stop - Not delirious)      4213
Unable to Assess (Stop)        1406
Yes (Continue)                  931
Name: count, dtype: int64
CAM-ICU Inattention
Yes                                               127712
No                                                 32737
Language Barrier                                    6232
Unable to Assess                                    4241
Preexisting Advanced Dementia                       2449
Yes (3 or more errors, then Continue)                615
No (less than 3 errors - Stop - Not delirious)       338
Name: count, dtype: int64
CAM-ICU Disorganized thinking
No                  19673
Yes                 12251
Unable to Assess      821
Name: count, dtype: int64
CAM-ICU Altered LOC
Yes    107833
No      21223
Name: count, dtype: int64
CAM-ICU RASS LOC
Yes    756
No     360
Name: count, dtype: int64


In [37]:
'''
delirious = Yes if 
(mental = True) AND (inattention = True) AND (LOC OR thinking = True)

which implies that delirious = No
if mental = False OR inattention = False OR (LOC = False AND thinking = False)
'''
query = f"""
SELECT
    hadm_id, time, COLUMNS('CAM-ICU'),
    coalesce("CAM-ICU Altered LOC", "CAM-ICU RASS LOC") AS loc,
    CASE
        WHEN ("CAM-ICU MS Change" LIKE '%Yes%') 
            AND ("CAM-ICU Inattention" LIKE '%Yes%') 
            AND (("CAM-ICU Disorganized thinking" LIKE '%Yes%') OR (loc LIKE '%Yes%'))
            THEN 'Positive'
        WHEN ("CAM-ICU MS Change" LIKE '%No%') 
            OR ("CAM-ICU Inattention" LIKE '%No%') 
            OR (("CAM-ICU Disorganized thinking" LIKE '%No%') AND (loc LIKE '%No%'))
            THEN 'Negative'
        ELSE NULL
        END AS cam_total
FROM cam_icu_w
"""
cam_icu_wc = con.execute(query).fetchdf()
cam_icu_wc

,hadm_id,time,CAM-ICU Altered LOC,CAM-ICU Disorganized thinking,CAM-ICU Inattention,CAM-ICU MS Change,CAM-ICU RASS LOC,loc,cam_total
0,28192338,2137-11-11 20:00:00,None,None,None,No,None,None,Negative
1,26789314,2161-04-15 00:00:00,None,None,None,No,None,None,Negative
2,21380829,2159-08-29 22:38:00,None,None,None,No,None,None,Negative
3,21380829,2159-08-29 08:00:00,None,None,None,No,None,None,Negative
4,21380829,2159-07-22 20:00:00,None,None,None,No,None,None,Negative
...,...,...,...,...,...,...,...,...,...
526884,20814033,2188-01-17 08:00:00,None,None,None,No,None,None,Negative
526885,23335631,2163-12-09 08:00:00,None,None,No,Yes,None,None,Negative
526886,23121624,2188-12-11 15:09:00,None,None,None,Unable to Assess,None,None,None
526887,22591561,2119-08-17 13:11:00,None,None,None,No,None,None,Negative


In [49]:
# unpivot (from wide to long) 
query = f"""
UNPIVOT (
    SELECT
        COLUMNS(* EXCLUDE ('loc'))
    FROM cam_icu_wc
)
ON COLUMNS('CAM-ICU|cam')
INTO
    NAME assessment_name
    VALUE categorical_value;
"""
cam_icu_wcl = con.execute(query).fetchdf()
cam_icu_wcl

,hadm_id,time,assessment_name,categorical_value
0,28192338,2137-11-11 20:00:00,CAM-ICU MS Change,No
1,28192338,2137-11-11 20:00:00,cam_total,Negative
2,26789314,2161-04-15 00:00:00,CAM-ICU MS Change,No
3,26789314,2161-04-15 00:00:00,cam_total,Negative
4,21380829,2159-08-29 22:38:00,CAM-ICU MS Change,No
...,...,...,...,...
1332353,23121624,2188-12-11 15:09:00,CAM-ICU MS Change,Unable to Assess
1332354,22591561,2119-08-17 13:11:00,CAM-ICU MS Change,No
1332355,22591561,2119-08-17 13:11:00,cam_total,Negative
1332356,26579973,2141-06-23 08:00:00,CAM-ICU MS Change,No


In [50]:
cam_icu_wcl.value_counts("assessment_name")

assessment_name
CAM-ICU MS Change                518056
cam_total                        477061
CAM-ICU Inattention              174324
CAM-ICU Altered LOC              129056
CAM-ICU Disorganized thinking     32745
CAM-ICU RASS LOC                   1116
Name: count, dtype: int64

In [71]:
query = f"""
SELECT
    hadm_id as hospitalization_id,
    time as recorded_dttm,
    CASE
        WHEN assessment_name = 'CAM-ICU MS Change' THEN 'cam_mental'
        WHEN assessment_name = 'cam_total' THEN 'cam_total'
        WHEN assessment_name = 'CAM-ICU Inattention' THEN 'cam_inattention'
        WHEN assessment_name = 'CAM-ICU Disorganized thinking' THEN 'cam_thinking'
        WHEN assessment_name in ('CAM-ICU RASS LOC', 'CAM-ICU Altered LOC') THEN 'cam_loc'
        ELSE NULL
        END AS assessment_category,
    CASE
        WHEN assessment_name = 'cam_total' THEN NULL
        ELSE assessment_name
        END AS assessment_name,
    categorical_value
FROM cam_icu_wcl
"""
cam_icu_wclc = con.execute(query).fetchdf()
cam_icu_wclc

,hospitalization_id,recorded_dttm,assessment_category,assessment_name,categorical_value
0,28192338,2137-11-11 20:00:00,cam_mental,CAM-ICU MS Change,No
1,28192338,2137-11-11 20:00:00,cam_total,None,Negative
2,26789314,2161-04-15 00:00:00,cam_mental,CAM-ICU MS Change,No
3,26789314,2161-04-15 00:00:00,cam_total,None,Negative
4,21380829,2159-08-29 22:38:00,cam_mental,CAM-ICU MS Change,No
...,...,...,...,...,...
1332353,23121624,2188-12-11 15:09:00,cam_mental,CAM-ICU MS Change,Unable to Assess
1332354,22591561,2119-08-17 13:11:00,cam_mental,CAM-ICU MS Change,No
1332355,22591561,2119-08-17 13:11:00,cam_total,None,Negative
1332356,26579973,2141-06-23 08:00:00,cam_mental,CAM-ICU MS Change,No


## Pain

In [72]:
pain = fetch_mimic_events([223795])

2025-01-20 23:49:52,462 - INFO - querying the d_items table to identify which event tables to be separately queried for 1 items
2025-01-20 23:49:52,468 - INFO - identified 1 event tables to be separately queried: ['chartevents']
2025-01-20 23:49:52,468 - INFO - fetching events from chartevents table for 1 items
2025-01-20 23:49:54,697 - INFO - fetched 1434168 events from chartevents table for 1 items
2025-01-20 23:49:54,698 - INFO - concatenated 1434168 events from 1 event tables


In [73]:
pain

,itemid,label,hadm_id,stay_id,time,value,valueuom
0,223795,Pain Assessment Method,29079034,39553978,2180-07-23 14:15:00,Patient Verbalized,None
1,223795,Pain Assessment Method,29079034,39553978,2180-07-23 18:23:00,Patient Verbalized,None
2,223795,Pain Assessment Method,29079034,39553978,2180-07-23 21:12:00,Patient Verbalized,None
3,223795,Pain Assessment Method,25860671,37081114,2150-11-06 08:00:00,Patient Verbalized,None
4,223795,Pain Assessment Method,25860671,37081114,2150-11-06 12:00:00,Patient Verbalized,None
...,...,...,...,...,...,...,...
1434163,223795,Pain Assessment Method,25252564,33057362,2190-09-20 20:00:00,Non-verbal Cues,None
1434164,223795,Pain Assessment Method,25252564,33057362,2190-09-21 00:00:00,Non-verbal Cues,None
1434165,223795,Pain Assessment Method,25252564,33057362,2190-09-21 04:00:00,Non-verbal Cues,None
1434166,223795,Pain Assessment Method,25252564,33057362,2190-09-13 08:00:00,Non-verbal Cues,None


In [74]:
pain.value_counts("value")

value
Patient Verbalized    788435
Non-verbal Cues       331848
Change in Vitals      159265
Grimace Scale         131896
FLACC                  17680
Not applicable          3167
PAINAD                  1121
Wong-Baker Scale         756
Name: count, dtype: int64

In [75]:
pain.query("value == 'PAINAD'")

,itemid,label,hadm_id,stay_id,time,value,valueuom
2495,223795,Pain Assessment Method,24332085,31392237,2136-11-27 08:14:00,PAINAD,None
2496,223795,Pain Assessment Method,24332085,31392237,2136-11-27 09:00:00,PAINAD,None
2497,223795,Pain Assessment Method,24332085,31392237,2136-11-27 13:00:00,PAINAD,None
3079,223795,Pain Assessment Method,20265910,33312557,2172-03-24 20:00:00,PAINAD,None
3080,223795,Pain Assessment Method,20265910,33312557,2172-03-24 22:45:00,PAINAD,None
...,...,...,...,...,...,...,...
1418286,223795,Pain Assessment Method,29175646,36709438,2175-08-04 20:08:00,PAINAD,None
1418287,223795,Pain Assessment Method,29175646,36709438,2175-08-05 00:25:00,PAINAD,None
1418288,223795,Pain Assessment Method,29175646,36709438,2175-08-05 04:17:00,PAINAD,None
1419727,223795,Pain Assessment Method,25488122,30047981,2141-12-17 10:42:00,PAINAD,None


In [76]:
pain2 = fetch_mimic_events([229687])
pain2.value_counts("value")

2025-01-20 23:51:56,670 - INFO - querying the d_items table to identify which event tables to be separately queried for 1 items
2025-01-20 23:51:56,672 - INFO - identified 1 event tables to be separately queried: ['chartevents']
2025-01-20 23:51:56,673 - INFO - fetching events from chartevents table for 1 items
2025-01-20 23:51:58,596 - INFO - fetched 499310 events from chartevents table for 1 items
2025-01-20 23:51:58,597 - INFO - concatenated 499310 events from 1 event tables


value
Patient Verbalized    401202
Non-verbal Cues        95263
Wong-Baker Scale        1917
---                      928
Name: count, dtype: int64

In [79]:
pain_level = fetch_mimic_events([223791])
pain_level.value_counts("value")

2025-01-20 23:53:35,860 - INFO - querying the d_items table to identify which event tables to be separately queried for 1 items
2025-01-20 23:53:35,863 - INFO - identified 1 event tables to be separately queried: ['chartevents']
2025-01-20 23:53:35,864 - INFO - fetching events from chartevents table for 1 items
2025-01-20 23:53:37,906 - INFO - fetched 1128488 events from chartevents table for 1 items
2025-01-20 23:53:37,906 - INFO - concatenated 1128488 events from 1 event tables


value
Unable to Score        314685
None                   294197
Moderate                98789
Mild to Moderate.       71069
Mild to Moderate        66483
Moderate to Severe      60771
Mild                    53309
Moderate to Severe.     52124
Severe                  46230
None to Mild            30945
Worst                   21973
Severe to Worse         17913
Name: count, dtype: int64

In [80]:
pain_level_resp = fetch_mimic_events([224409])
pain_level_resp.value_counts("value")

2025-01-20 23:54:37,953 - INFO - querying the d_items table to identify which event tables to be separately queried for 1 items
2025-01-20 23:54:37,955 - INFO - identified 1 event tables to be separately queried: ['chartevents']
2025-01-20 23:54:37,955 - INFO - fetching events from chartevents table for 1 items
2025-01-20 23:54:39,850 - INFO - fetched 664970 events from chartevents table for 1 items
2025-01-20 23:54:39,851 - INFO - concatenated 664970 events from 1 event tables


value
Unable to Score        226452
None                   175417
Mild                    65901
Mild to Moderate        53851
None to Mild            44704
Mild to Moderate.       34812
Moderate                30792
Moderate to Severe      12500
Moderate to Severe.      8756
Severe                   6223
Worst                    3186
Severe to Worse          2376
Name: count, dtype: int64

## CPOT

In [ ]:
cpot = fetch_mimic_events([229698, 229696, 229691, 229695])

## SBT

In [ ]:
sbt_items = search_mimic_items(kw="sbt")

In [7]:
sbt_items.head()

,kw,itemid,label,abbreviation,linksto,category,unitname,param_type,count,value_instances
2,sbt,224715,SBT Started,SBT Started,chartevents,Respiratory,None,Text,58874,"Yes: 39729, No: 19145"
1,sbt,224717,SBT Successfully Completed,SBT Successfully Completed,chartevents,Respiratory,None,Text,35695,"Yes: 29610, No: 6085"
0,sbt,224833,SBT Deferred,SBT Deferred,chartevents,Respiratory,None,Text,23285,"Unable to perform RSBI: 16936, Chronic vent de..."
3,sbt,224716,SBT Stopped,SBT Stopped,chartevents,Respiratory,None,Text,6250,"RR > 35 for > 5 min: 2872, Accessory muscle us..."


In [9]:
sbt_items["value_instances"].tolist()

['Yes: 39729, No: 19145',
 'Yes: 29610, No: 6085',
 'Unable to perform RSBI: 16936, Chronic vent dependent patient: 3540, Pending procedure : 2460, RSBI > 105: 349',
 'RR > 35 for > 5 min: 2872, Accessory muscle use: 1660, SpO2 < 90% for > 2 min: 966, BP > 180 or < 90: 396, HR > 140: 218, Arrythmia: 138']

In [5]:
sbt_events = fetch_mimic_events([224717, 224833, 224716]) 
sbt_events.head()

2025-04-22 23:20:00,226 - INFO - querying the d_items table to identify which event tables to be separately queried for 3 items
2025-04-22 23:20:00,233 - INFO - identified 1 event tables to be separately queried: ['chartevents']
2025-04-22 23:20:00,234 - INFO - fetching events from chartevents table for 3 items
2025-04-22 23:20:02,046 - INFO - fetched 65230 events from chartevents table for 3 items
2025-04-22 23:20:02,047 - INFO - concatenated 65230 events from 1 event tables


,itemid,label,hadm_id,stay_id,time,value,valueuom
0,224717,SBT Successfully Completed,29242151,30213599,2159-02-22 05:00:00,Yes,None
1,224717,SBT Successfully Completed,29242151,30213599,2159-02-21 04:25:00,Yes,None
2,224717,SBT Successfully Completed,22081550,35009126,2186-11-17 04:00:00,Yes,None
3,224717,SBT Successfully Completed,22081550,35009126,2186-11-14 04:00:00,Yes,None
4,224833,SBT Deferred,22081550,35009126,2186-11-13 04:00:00,Unable to perform RSBI,None


In [6]:
# check there is no NA
sbt_events.value_counts("value", dropna=False)

value
Yes                               29610
Unable to perform RSBI            16936
No                                 6085
Chronic vent dependent patient     3540
RR > 35 for > 5 min                2872
Pending procedure                  2460
Accessory muscle use               1660
SpO2 < 90% for > 2 min              966
BP > 180 or < 90                    396
RSBI > 105                          349
HR > 140                            218
Arrythmia                           138
Name: count, dtype: int64

In [7]:
sbt_id_to_category_mapper = {
    224717: "sbt_delivery_pass_fail",
    224833: "sbt_fail_reason",
    224716: "sbt_fail_reason"
}

In [ ]:
sbt_events.value_counts("value")

In [8]:
sbt_events_translated = sbt_events.assign(
    assessment_category = lambda x: x["itemid"].map(sbt_id_to_category_mapper)
)
sbt_events_translated.head()

,itemid,label,hadm_id,stay_id,time,value,valueuom,assessment_category
0,224717,SBT Successfully Completed,29242151,30213599,2159-02-22 05:00:00,Yes,None,sbt_delivery_pass_fail
1,224717,SBT Successfully Completed,29242151,30213599,2159-02-21 04:25:00,Yes,None,sbt_delivery_pass_fail
2,224717,SBT Successfully Completed,22081550,35009126,2186-11-17 04:00:00,Yes,None,sbt_delivery_pass_fail
3,224717,SBT Successfully Completed,22081550,35009126,2186-11-14 04:00:00,Yes,None,sbt_delivery_pass_fail
4,224833,SBT Deferred,22081550,35009126,2186-11-13 04:00:00,Unable to perform RSBI,None,sbt_fail_reason


In [26]:
query = f"""
SELECT
    CAST(hadm_id as VARCHAR) as hospitalization_id,
    time as recorded_dttm,
    label as assessment_name,
    assessment_category,
    CAST(NULL as FLOAT) as numerical_value,
    CASE WHEN assessment_category = 'sbt_delivery_pass_fail' 
        THEN (CASE WHEN value = 'Yes' THEN 'Pass' 
            WHEN value = 'No' THEN 'Fail' 
            ELSE NULL END) 
        ELSE NULL
        END AS categorical_value,
    CASE WHEN assessment_category = 'sbt_fail_reason' 
        THEN value ELSE NULL
        END AS text_value
FROM sbt_events_translated
"""
sbt_events_renamed_reordered = con.execute(query).fetchdf()
sbt_events_renamed_reordered.head()

,hospitalization_id,recorded_dttm,assessment_name,assessment_category,numerical_value,categorical_value,text_value
0,29242151,2159-02-22 05:00:00,SBT Successfully Completed,sbt_delivery_pass_fail,NaN,Pass,None
1,29242151,2159-02-21 04:25:00,SBT Successfully Completed,sbt_delivery_pass_fail,NaN,Pass,None
2,22081550,2186-11-17 04:00:00,SBT Successfully Completed,sbt_delivery_pass_fail,NaN,Pass,None
3,22081550,2186-11-14 04:00:00,SBT Successfully Completed,sbt_delivery_pass_fail,NaN,Pass,None
4,22081550,2186-11-13 04:00:00,SBT Deferred,sbt_fail_reason,NaN,None,Unable to perform RSBI


In [21]:
sbt_events_renamed_reordered.dtypes

hospitalization_id             object
recorded_dttm          datetime64[us]
assessment_name                object
assessment_category            object
numerical_value               float32
categorical_value              object
text_value                     object
dtype: object

# Validate

In [5]:
pa = pd.read_parquet(clif_table_pathfinder("patient_assessments"))

In [7]:
pa.value_counts(["assessment_name", "assessment_category"], dropna=False)

assessment_name                                                 assessment_category   
gcs_verbal                                                      gcs_verbal                2217787
gcs_motor                                                       gcs_motor                 2217787
gcs_eyes                                                        gcs_eye                   2217787
gcs                                                             gcs_total                 2217787
Richmond-RAS Scale                                              RASS                      1656001
Braden Sensory Perception                                       braden_sensory            1068778
Braden Moisture                                                 braden_moisture           1068127
Braden Activity                                                 braden_activity           1067495
Braden Mobility                                                 braden_mobility           1067493
Braden Nutrition               